# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [18]:
# import libraries
import sys
import pandas as pd
import numpy as np
import nltk
import re
from nltk.tokenize import word_tokenize, WhitespaceTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from sqlalchemy import create_engine
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import pickle

In [2]:
nltk.download(['punkt', 'wordnet','stopwords'])

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [3]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql_table('MessageCategories', engine)
df.drop('original', axis=1, inplace=True) # A lot of NaN values for this column do dropping it
df.dropna(how='any', inplace=True)
X = df['message']
Y = df.iloc[:,3:]
category_names = Y.columns

### 2. Write a tokenization function to process your text data

In [4]:
def tokenize(text):
    '''

    input:

        text: Text data for tokenization.

    output:

        tokenized_clean: List of results after tokenization.

    '''
    # Normalize Text
    text = re.sub(r"[^a-zA-Z0-9]", ' ', text.lower())

    # Tokenize
    words = word_tokenize(text)

    # Remove Stopwords
    stopwordlist = list(set(stopwords.words('english')))
    words = [x for x in words if x not in stopwordlist]

    # Lemmatize
    lemmatizer = WordNetLemmatizer()
    lemmatized = [lemmatizer.lemmatize(x, pos='n').strip() for x in words]
    tokenized_clean = [lemmatizer.lemmatize(x, pos='v').strip() for x in lemmatized]
    
    return tokenized_clean

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [70]:
pipeline = Pipeline([('vect', CountVectorizer(tokenizer=tokenize)),
                      ('tfidf', TfidfTransformer()),
                      ('clf', MultiOutputClassifier(RandomForestClassifier()))])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [71]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2)
pipeline.fit(X_train, Y_train)

Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [72]:
# get predictions 
Y_pred = pipeline.predict(X_test)

for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

        0.0       0.34      0.16      0.22      1212
        1.0       0.78      0.91      0.84      3992
        2.0       0.50      0.03      0.05        38

avg / total       0.67      0.73      0.69      5242

Accuracy of                   related: 0.73
Category: request 
              precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      4291
        1.0       0.43      0.08      0.13       951

avg / total       0.76      0.81      0.76      5242

Accuracy of                   request: 0.81
Category: offer 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5242

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

        0.0       0.59      0.81  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 6. Improve your model
Use grid search to find better parameters. 

In [79]:
parameters = {'clf__estimator__min_samples_split': [2, 4],
              #'clf__estimator__max_features': ['log2', 'auto', 'sqrt', None],
              #'clf__estimator__criterion': ['gini', 'entropy'],
              #'clf__estimator__max_depth': [None, 25, 50, 100, 150, 200],
             }

cv = GridSearchCV(estimator=pipeline, param_grid=parameters)

In [80]:
cv.fit(X_train, Y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(memory=None,
     steps=[('vect', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip...oob_score=False, random_state=None, verbose=0,
            warm_start=False),
           n_jobs=1))]),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'clf__estimator__min_samples_split': [2, 4]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [83]:
Y_pred = pipeline.predict(X_test)

for i in range(len(category_names)):
    print("Category:", category_names[i],"\n", classification_report(Y_test.iloc[:, i].values, Y_pred[:, i]))
    print('Accuracy of %25s: %.2f' %(category_names[i], accuracy_score(Y_test.iloc[:, i].values, Y_pred[:,i])))

Category: related 
              precision    recall  f1-score   support

        0.0       0.34      0.16      0.22      1212
        1.0       0.78      0.91      0.84      3992
        2.0       0.50      0.03      0.05        38

avg / total       0.67      0.73      0.69      5242

Accuracy of                   related: 0.73
Category: request 
              precision    recall  f1-score   support

        0.0       0.83      0.98      0.90      4291
        1.0       0.43      0.08      0.13       951

avg / total       0.76      0.81      0.76      5242

Accuracy of                   request: 0.81
Category: offer 
              precision    recall  f1-score   support

        0.0       1.00      1.00      1.00      5218
        1.0       0.00      0.00      0.00        24

avg / total       0.99      1.00      0.99      5242

Accuracy of                     offer: 1.00
Category: aid_related 
              precision    recall  f1-score   support

        0.0       0.59      0.81  

/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

### 9. Export your model as a pickle file

In [84]:
pickle.dump(cv, open('disaster_response_model.sav', 'wb'))

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.